# Herramientas de optimización matemática en Python  
Ángel Manuel González Rueda  
<img src="img/modes.jpg" alt="drawing" width="150" align="left"/>

# PYOMO

Web de referencia: http://www.pyomo.org/  
Documentación: http://www.pyomo.org/documentation, https://pyomo.readthedocs.io/en/stable/index.html  
Presentación introducción a Pyomo: https://software.sandia.gov/downloads/pub/pyomo/Pyomo-Workshop-Summer-2018.pdf

* Es un **software libre** que proporciona el uso de lenguaje de modelado algebraico para formular problemas de programación matemática.
* Los objetos de Pyomo, al estar embebidos en **Python**, permiten emplear un completo lenguaje de programación de alto nivel con acceso a un rico conjunto de librerías de apoyo. 
* Pyomo soporta la formulación de un gran rango de tipos de problema:
    * Programación lineal (entera-mixta)
    * Programación cuadrática (entera-mixta)
    * Programación no lineal (entera-mixta)
    * ...
* Incluye un paquete (PySP) para implementar problemas de programación estocástica.
* Permite conectarse con un gran conjunto de solvers.

![title](img/pyomo.png)

## Concrete Model
- Primero hay que definir los datos, y luego el modelo.
- Contrucción en 1 paso.
- Todos los datos deben de estar presentes antes que Python empiece a procesar el modelo.
- Pyomo irá contruyendo cada componente en el momento que se van definiendo.
- Proceso lógico senccilo, facilita realizar 'scripting'.

### Simple modeling Example: Classic Knapsack Problem
- Dado un conjunto de objetos A, con cierto peso y beneficio
- Objetivo: seleccionar un subconjunto de esos objetos
    - Maximizando el beneficio
    - Respectando el peso máximo ($W_{max}$)

|Objetos(A)      | Peso(w)      | Beneficio(b) |
| :------------  | :----------: | -----------: |
| Tornillo       | 5            | 8            |
| Candado        | 7            | 3            |
| Destornillado  | 4            | 6            |
| Toalla         | 3            | 11           |

\begin{align*}
\textrm{Maximizar}\ \ \     & \sum_{i\in A}b_{i}x_{i}\\
\textrm{sujeto a}\ \ \      & \sum_{i\in A}w_{i}x_{i}\leq W_{max}\\
                 \ \ \      & x_{i}\in \{0,1\} \, \, \forall i \in A\\                     
\end{align*}

Elementos principales a considerar a la hora de crear el modelo con Pyomo:
    - Con respecto a los datos:
        - Conjuntos
        - Parámetros
    - Con respecto al modelo:
        - Variables
        - Función objetivo
        - Restricciones

### Importamos el paquete

In [109]:
import pyomo.environ as pe

### Definimos los datos de la instancia que nos interesa resolver

In [110]:
A = ['tornillo','candado','destornillador','toalla']
b = {'tornillo': 8,
     'candado':3,
     'destornillador':6,
     'toalla':11
    }
w = {'tornillo': 5,
     'candado':7,
     'destornillador':4,
     'toalla':3
    }
W_max = 14

### Creamos una instancia concreta del modelo
* Los modelos concretos se construyen inmediatamente
* Los datos deben estar definifos a la hora de contruir el modelo

In [117]:
model = pe.ConcreteModel()

### Definimos las variables
* La debemos definir como un atributo del objeto model, y el nombre debe ser único dentro del modelo considerado

In [118]:
model.x = pe.Var(A,within=pe.Binary)

In [119]:
help(pe.Var)

Help on class Var in module pyomo.core.base.var:

class Var(pyomo.core.base.indexed_component.IndexedComponent)
 |  Var(*args, **kwds)
 |  
 |  A numeric variable, which may be defined over an index.
 |  
 |  Args:
 |      domain (Set or function, optional): A Set that defines valid
 |          values for the variable (e.g., `Reals`, `NonNegativeReals`,
 |          `Binary`), or a rule that returns Sets.  Defaults to `Reals`.
 |      within (Set or function, optional): An alias for `domain`.
 |      bounds (tuple or function, optional): A tuple of (lower, upper)
 |          bounds for the variable, or a rule that returns tuples.
 |          Defaults to (None, None).
 |      initialize (float or function, optional): The initial value for
 |          the variable, or a rule that returns initial values.
 |      rule (float or function, optional): An alias for `initialize`.
 |      dense (bool, optional): Instantiate all elements from
 |          `index_set()` when constructing the Var (Tr

### Definimos la función objetivo

In [120]:
model.benefit = pe.Objective(
    expr=sum(b[i]*model.x[i] for i in A),
    sense= pe.maximize)

In [121]:
help(pe.Objective)

Help on class Objective in module pyomo.core.base.objective:

class Objective(pyomo.core.base.indexed_component.ActiveIndexedComponent)
 |  Objective(*args, **kwds)
 |  
 |  This modeling component defines an objective expression.
 |  
 |  Note that this is a subclass of NumericValue to allow
 |  objectives to be used as part of expressions.
 |  
 |  Constructor arguments:
 |      expr            
 |          A Pyomo expression for this objective
 |      rule            
 |          A function that is used to construct objective expressions
 |      sense           
 |          Indicate whether minimizing (the default) or maximizing
 |      doc             
 |          A text string describing this component
 |      name            
 |          A name for this component
 |  
 |  Public class attributes:
 |      doc             
 |          A text string describing this component
 |      name            
 |          A name for this component
 |      active          
 |          A boolean

### Definimos las restricciones

In [123]:
model.weight = pe.Constraint(
    expr=sum(w[i]*model.x[i] for i in A)<=W_max )

# Otra forma análoga, especificando cota inferior y superior de la restricción
# model.weight = pe.Constraint(
#    expr=(None,sum(w[i]*model.x[i] for i in A),W_max) )

    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [124]:
help(pe.Constraint)

Help on class Constraint in module pyomo.core.base.constraint:

class Constraint(pyomo.core.base.indexed_component.ActiveIndexedComponent)
 |  Constraint(*args, **kwds)
 |  
 |  This modeling component defines a constraint expression using a
 |  rule function.
 |  
 |  Constructor arguments:
 |      expr 
 |          A Pyomo expression for this constraint
 |      rule 
 |          A function that is used to construct constraint expressions
 |      doc 
 |          A text string describing this component
 |      name 
 |          A name for this component
 |  
 |  Public class attributes:
 |      doc 
 |          A text string describing this component
 |      name 
 |          A name for this component
 |      active  
 |          A boolean that is true if this component will be used to 
 |          construct a model instance
 |      rule 
 |         The rule used to initialize the constraint(s)
 |  
 |  Private class attributes:
 |      _constructed        
 |          A boolean that 

### Creamos la factoría del solver que queremos utilizar

In [125]:
opt = pe.SolverFactory('glpk')

### Resolvemos el problema

In [126]:
result_obj = opt.solve(model,tee=True) # tee=True visualiza la salida del solver

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\User\AppData\Local\Temp\tmpe2nrfgwc.glpk.raw --wglp C:\Users\User\AppData\Local\Temp\tmp6l28tpab.glpk.glp
 --cpxlp C:\Users\User\AppData\Local\Temp\tmpdzx8bysc.pyomo.lp
Reading problem data from 'C:\Users\User\AppData\Local\Temp\tmpdzx8bysc.pyomo.lp'...
C:\Users\User\AppData\Local\Temp\tmpdzx8bysc.pyomo.lp:28: warning: lower bound of variable 'x1' redefined
C:\Users\User\AppData\Local\Temp\tmpdzx8bysc.pyomo.lp:28: warning: upper bound of variable 'x1' redefined
2 rows, 5 columns, 5 non-zeros
4 integer variables, all of which are binary
32 lines were read
Writing problem data to 'C:\Users\User\AppData\Local\Temp\tmp6l28tpab.glpk.glp'...
22 lines were written
GLPK Integer Optimizer, v4.65
2 rows, 5 columns, 5 non-zeros
4 integer variables, all of which are binary
Preprocessing...
1 constraint coefficient(s) were reduced
1 row, 4 columns, 4 non-zeros
4 integer variables, all of which are binary

### Imprimimos los resultados

In [127]:
model.pprint()

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['candado', 'destornillador', 'toalla', 'tornillo']

1 Var Declarations
    x : Size=4, Index=x_index
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
               candado :     0 :   0.0 :     1 : False : False : Binary
        destornillador :     0 :   1.0 :     1 : False : False : Binary
                toalla :     0 :   1.0 :     1 : False : False : Binary
              tornillo :     0 :   1.0 :     1 : False : False : Binary

1 Objective Declarations
    benefit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[tornillo] + 3*x[candado] + 6*x[destornillador] + 11*x[toalla]

1 Constraint Declarations
    weight : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                             : Upper : Active
        None :  -Inf : 5*x[tornillo] 

In [128]:
model.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
               candado :     0 :   0.0 :     1 : False : False : Binary
        destornillador :     0 :   1.0 :     1 : False : False : Binary
                toalla :     0 :   1.0 :     1 : False : False : Binary
              tornillo :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    benefit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    weight : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0


### Imprimimos los valores de cada variables de forma explícita

In [129]:
#
# Print values for each variable explicitly
#
print("#############################################")
print("Print values for each variable explicitly")
for i in model.x:
  print(str(model.x[i]), model.x[i].value)
print("")

#
# Print values for all variables
#
print("#############################################")
print("Print values for all variables")
for v in model.component_data_objects(pe.Var):
  print(str(v), v.value)

#############################################
Print values for each variable explicitly
x[tornillo] 1.0
x[candado] 0.0
x[destornillador] 1.0
x[toalla] 1.0

#############################################
Print values for all variables
x[tornillo] 1.0
x[candado] 0.0
x[destornillador] 1.0
x[toalla] 1.0


## Un ejemplo un poco más completo: Warehouse Location
* Determinar el conjunto de $P$ almacenes de entre el conjunto total $W$ de posibles almacenes minimizando los costes de servicio de todos los clientes $C$.
* $d_{w,c}$ representa el coste de servir al cliente $c$ desde el almacén ubicado en $w$
* Variables del problema:
    * $y_{w}\in {0,1}$: determina si el almacén ubicado en $w$ es seleccionado o no.
    * $x_{w,c}$: porcentaje de demanda del cliente $c$ satisfecha por el almacén $w$.
    
\begin{align*}
\textrm{Maximizar}\ \ \     & \sum_{w\in W, c \in C}d_{w,c}x_{w,c}\\
\textrm{sujeto a}\ \ \      & \sum_{w\in W}x_{w,c}= 1 \, \, \forall c \in C \text{(Todos los clientes son servidos)}\\
                 \ \ \     & x_{w,c}\leq y_{w}  \, \, \forall w\in W, \forall c \in C \text{(Si el cliente es servido por $w$, debe ser seleccionado)}\\  
                 \ \ \     & \sum_{w\in W}y_{w}= P \, \,\text{(Seleccionar $P$ almacenes)}\\
                 \ \ \     & 0\leq x\leq 1, y\in \{0,1\}^{|W|} \, \,\text{(Seleccionar $P$ almacenes)}\\
\end{align*}

La principal diferencia con el ejemplo anterior es que ahora necesatiamos definir **restricciones sobre un conjunto**. Para ello emplearemos el concepto de **regla** para definir la expresión lineal correspondiente.

### Datos
![title](img/warehouse_data.png)

In [130]:
W = ['Harlingen', 'Memphis', 'Ashland']
C = ['NYC', 'LA', 'Chicago', 'Houston']
d = {('Harlingen', 'NYC'): 1956,
     ('Harlingen', 'LA'): 1606,
     ('Harlingen', 'Chicago'): 1410,
     ('Harlingen', 'Houston'): 330,
     ('Memphis', 'NYC'): 1096,
     ('Memphis', 'LA'): 1792,
     ('Memphis', 'Chicago'): 531,
     ('Memphis', 'Houston'): 567,
     ('Ashland', 'NYC'): 485,
     ('Ashland', 'LA'): 2322,
     ('Ashland', 'Chicago'): 324,
     ('Ashland', 'Houston'): 1236
    }
P = 2

#### Definimos el modelo y las variables:

In [131]:
model = pe.ConcreteModel(name="(WL)")

model.x = pe.Var(W,C, bounds=(0,1))
model.y = pe.Var(W, within=pe.Binary)

### Función objetivo: en este caso la definiremos empleando una regla, en lugar de generando la expresión lineal directamente

In [132]:
def obj_rule(m):
    return sum(d[w,c]*m.x[w,c] for w in W for c in C)
#model.obj = pe.Objective(expr=sum(d[w,c]*m.x[w,c] for w in W for c in C))
model.obj = pe.Objective(rule=obj_rule)

### Restricción: $\sum_{w\in W}x_{w,c}= 1 \, \, \forall c \in C \text{(Todos los clientes son servidos)}$

In [133]:
def one_per_cust_rule(m, c):
    return sum(m.x[w,c] for w in W) == 1
model.one_per_cust = pe.Constraint(C, rule=one_per_cust_rule)

### Restricción: $x_{w,c}\leq y_{w}  \, \, \forall w\in W, \forall c \in C \text{(Si el cliente es servido por $w$, debe ser seleccionado)}$

In [134]:
def warehouse_active_rule(m, w, c):
    return m.x[w,c] <= m.y[w]
model.warehouse_active = pe.Constraint(W, C, rule=warehouse_active_rule)

### Restricción: $\sum_{w\in W}y_{w}= P \, \,\text{(Seleccionar $P$ almacenes)}$

In [135]:
def num_warehouses_rule(m):
    return sum(m.y[w] for w in W) <= P
model.num_warehouses = pe.Constraint(rule=num_warehouses_rule)

### Especificamos solver y resolvemos

In [136]:
pe.SolverFactory('glpk').solve(model)
model.pprint()

8 Set Declarations
    one_per_cust_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['Chicago', 'Houston', 'LA', 'NYC']
    warehouse_active_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=False, Bounds=None
        Virtual
    warehouse_active_index_0 : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['Ashland', 'Harlingen', 'Memphis']
    warehouse_active_index_1 : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['Chicago', 'Houston', 'LA', 'NYC']
    x_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['Ashland', 'Harlingen', 'Memphis']
    x_index_1 : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['Chicago', 'Houston', 'LA', 'NYC']
    y_index : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['Ashland', 'Harlingen', 'Memph

In [137]:
model.display()

Model (WL)

  Variables:
    x : Size=12, Index=x_index
        Key                      : Lower : Value : Upper : Fixed : Stale : Domain
          ('Ashland', 'Chicago') :     0 :   1.0 :     1 : False : False :  Reals
          ('Ashland', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
               ('Ashland', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
              ('Ashland', 'NYC') :     0 :   1.0 :     1 : False : False :  Reals
        ('Harlingen', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
        ('Harlingen', 'Houston') :     0 :   1.0 :     1 : False : False :  Reals
             ('Harlingen', 'LA') :     0 :   1.0 :     1 : False : False :  Reals
            ('Harlingen', 'NYC') :     0 :   0.0 :     1 : False : False :  Reals
          ('Memphis', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
          ('Memphis', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
               ('Memphis', 'LA') :     0 :

# Abstract models
- Primero el modelo, y luego los datos.
- Contrucción en 2 pasos.
- Pyomo va almacenando las declaraciones básicas del modelo, pero no contruye de forma instantánea los objetos.
- En el momento de la creación del modelo, los datos son aplicados al conjunto de declaraciones abstractas para crear una instancia concreta del modelo.
- Favorece modelado más genérico y reutilización del modelo.
- Familiar para la gente acostumbrada a utilizar AMPL.

## Abstract Knapsack example

### Modelo (guardado en .mod)

In [138]:
model = pe.AbstractModel()

# Datos de forma abstracta (como conjuntos y parámetros)
model.ITEMS = pe.Set()
model.v = pe.Param( model.ITEMS, within=pe.PositiveReals )
model.w = pe.Param( model.ITEMS, within=pe.PositiveReals )
model.W_max = pe.Param( within=pe.PositiveReals )

# Variables
model.x = pe.Var( model.ITEMS, within=pe.Binary )

# Función objetivo
def value_rule(model):
    return sum( model.v[i]*model.x[i] for i in model.ITEMS )
model.value = pe.Objective( rule=value_rule, sense=pe.maximize )

# Restricción
def weight_rule(model):
    return sum( model.w[i]*model.x[i] for i in model.ITEMS )<= model.W_max
model.weight = pe.Constraint( rule=weight_rule )

In [139]:
model.pprint()

1 Set Declarations
    ITEMS : Dim=0, Dimen=1, Size=0, Domain=None, Ordered=False, Bounds=None
        Not constructed

3 Param Declarations
    W_max : Size=0, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Not constructed
    v : Size=0, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Not constructed
    w : Size=0, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Not constructed

1 Var Declarations
    x : Size=0, Index=ITEMS
        Not constructed

1 Objective Declarations
    value : Size=0, Index=None, Active=True
        Not constructed

1 Constraint Declarations
    weight : Size=0, Index=None, Active=True
        Not constructed

7 Declarations: ITEMS v w W_max x value weight


In [140]:
help(model.create_instance)

Help on method create_instance in module pyomo.core.base.PyomoModel:

create_instance(filename=None, data=None, name=None, namespace=None, namespaces=None, profile_memory=0, report_timing=False, **kwds) method of pyomo.core.base.PyomoModel.AbstractModel instance
    Create a concrete instance of an abstract model, possibly using data
    read in from a file.
    
    Parameters
    ----------
    filename: `str`, optional           
        The name of a Pyomo Data File that will be used to load data into 
        the model.
    data: `dict`, optional
        A dictionary containing initialization data for the model to be 
        used if there is no filename
    name: `str`, optional
        The name given to the model.
    namespace: `str`, optional          
        A namespace used to select data.
    namespaces: `list`, optional   
        A list of namespaces used to select data.
    profile_memory: `int`, optional    
        A number that indicates the profiling level.
    repo

### Instanciamos modelo abstracto usando un diccionario de Python  
https://pyomo.readthedocs.io/en/stable/working_abstractmodels/data/raw_dicts.html

In [141]:
data_input = {None:{
    'ITEMS': {None:['tornillo','candado','destornillador','toalla']}, 
    'v':{'tornillo': 8,'candado':3,'destornillador':6,'toalla':11},
    'w': {'tornillo': 5,'candado':7,'destornillador':4,'toalla':3},
    'W_max':{None: 14},
}}

In [142]:
instance = model.create_instance(data=data_input)
type(instance)

pyomo.core.base.PyomoModel.ConcreteModel

In [143]:
print('Model is contructed :',model.is_constructed())
print('Instance is contructed :',instance.is_constructed())

Model is contructed : False
Instance is contructed : True


In [144]:
instance.pprint()

1 Set Declarations
    ITEMS : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['candado', 'destornillador', 'toalla', 'tornillo']

3 Param Declarations
    W_max : Size=1, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Key  : Value
        None :    14
    v : Size=4, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Key            : Value
               candado :     3
        destornillador :     6
                toalla :    11
              tornillo :     8
    w : Size=4, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Key            : Value
               candado :     7
        destornillador :     4
                toalla :     3
              tornillo :     5

1 Var Declarations
    x : Size=4, Index=ITEMS
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
               candado :     0 :  None :     1 : False :  True : Binary
        destornillador :     0 :  None 

In [145]:
pe.SolverFactory('glpk').solve(instance)
instance.display()

Model unknown

  Variables:
    x : Size=4, Index=ITEMS
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
               candado :     0 :   0.0 :     1 : False : False : Binary
        destornillador :     0 :   1.0 :     1 : False : False : Binary
                toalla :     0 :   1.0 :     1 : False : False : Binary
              tornillo :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    value : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    weight : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0


### Instanciamos modelo abstracto usando un archivo .dat  
* Se puede hacer utilizando lenguaje Python
* Se puede hacer una llamada a sistema a pyomo especificando model.py y datos.dat

set ITEMS := tornillo candado destornillador toalla ;
    
param: v w :=
    tornillo 8 5
    candado 3 7
    destornillador 6 4
    toalla 11 3;
    
param W_max := 14;

In [146]:
import os
print("Current Working Directory " , os.getcwd())

Current Working Directory  C:\Users\User\Documents\Congresos 2019\Curso de Python\Notebooks Angel


### Resolvemos utilizando API de Python

In [147]:
instance2 = model.create_instance(filename='datos_input.dat')
pe.SolverFactory('glpk').solve(instance)
instance.display()

Model unknown

  Variables:
    x : Size=4, Index=ITEMS
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
               candado :     0 :   0.0 :     1 : False : False : Binary
        destornillador :     0 :   1.0 :     1 : False : False : Binary
                toalla :     0 :   1.0 :     1 : False : False : Binary
              tornillo :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    value : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    weight : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0


### Resolvemos llamando a Pyomo por sistema de comandos

In [148]:
os.system("pyomo solve --solver=glpk model_knapsack.py datos_input.dat > output_file.txt")

1